In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import metrics


In [ ]:
SurfaceData=pd.read_csv('SurfaceData_SUM.csv',delimiter=";") # load training database

In [ ]:
for i in range(len(SurfaceData)):
    SurfaceData["bins"][i]=np.fromstring(SurfaceData["bins"][i][1:-2],sep=',')
    SurfaceData["n0"][i]=np.fromstring(SurfaceData["n0"][i][1:-2],sep=',')
    SurfaceData["Qquerys"][i]=np.fromstring(SurfaceData["Qquerys"][i][1:-2],sep=',')
    SurfaceData["PSquerys"][i]=np.fromstring(SurfaceData["PSquerys"][i][1:-2],sep=',')

In [ ]:
SurfaceData_Simulated=SurfaceData[SurfaceData["DU"].notna()]# Exclude not simulated samples
SurfaceData_Simulated=SurfaceData_Simulated[SurfaceData["DU"]>=6] # Exclude transitionally rough samples
SurfaceData_Pool=SurfaceData[SurfaceData["DU"].isna()]# Put not simulated samples to repository
ListOfLabledID=SurfaceData_Simulated["Surface_ID"].to_numpy()
krTrain=SurfaceData_Simulated["kr"].to_numpy()
SurfacePoolID=SurfaceData_Pool["Surface_ID"].to_numpy()

Iteration_num=50 # number of NN members
SurfaceTrain=[]
SurfacePool=[]

In [ ]:
for i in ListOfLabledID.astype(int): #Formulate input vectors of trainning data
    SurfaceTrain_row=[SurfaceData.iloc[i].kt/SurfaceData.iloc[i].K_99,SurfaceData.iloc[i].lambda0_k99,SurfaceData.iloc[i].lambda1_k99]
    SurfaceTrain_row.extend(SurfaceData.iloc[i].n0)
    SurfaceTrain_row.extend(SurfaceData.iloc[i].PSquerys)
    SurfaceTrain.append(SurfaceTrain_row)

In [ ]:
for i in SurfacePoolID.astype(int): #Formulate input vectors of rest data in repository 
    SurfacePool_row=[SurfaceData.iloc[i].kt/SurfaceData.iloc[i].K_99,SurfaceData.iloc[i].lambda0_k99,SurfaceData.iloc[i].lambda1_k99]
    SurfacePool_row.extend(SurfaceData.iloc[i].n0)
    SurfacePool_row.extend(SurfaceData.iloc[i].PSquerys)
    SurfacePool.append(SurfacePool_row)

In [ ]:
SurfaceTrain=np.array(SurfaceTrain)
SurfacePool=np.array(SurfacePool)

In [ ]:
def lr_scheduler(epoch, learning_rate):
    #lr = learning_rate
    if epoch == 1000:
        learning_rate = learning_rate*0.5
        return learning_rate
    elif epoch == 1500:
        learning_rate = learning_rate*0.5
        return learning_rate
    elif epoch == 1800:
        learning_rate = learning_rate*0.5
        return learning_rate
    else:
        #learning_rate = lr
        return learning_rate
    
new_lr = LearningRateScheduler(lr_scheduler, verbose=0)
def build_model(space):
    model = Sequential()
    input_shape = 63
    if space[3]=='leakyrelu':
        model.add(Dense(int(space[0]),input_shape=(input_shape,),activation=LeakyReLU(alpha=0.1),kernel_regularizer=regularizers.l2(space[5])))
    else:
        model.add(Dense(int(space[0]),input_shape=(input_shape,),activation=Activation(space[3]),kernel_regularizer=regularizers.l2(space[5])))
    for i in range(2):
        model.add(Dense(int(space[i+1]),kernel_regularizer=regularizers.l2(space[5])))
        if space[3] == 'leakyrelu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(Activation(space[3]))
    # Add output layer
    model.add(Dense(1,kernel_regularizer=regularizers.l2(space[5])))
    if space[3] == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.1))
    else:
        model.add(Activation(space[3]))
    # Compile the model
    model.compile(optimizer=Adam(space[4]), loss=MeanSquaredError(),
                  metrics=[metrics.MeanSquaredError(),metrics.MeanAbsolutePercentageError(name="MAPE")])
    return model

In [ ]:
predictions=[]
losshistory=[]
vallosshistory=[]
best_params_csv=pd.read_csv('./Hyperparameters.csv',sep=";") # read hyperparameters
for i in range(Iteration_num): # iteratively train NN members
    p = np.random.permutation(len(SurfaceTrain)) # Schuffle training data
    SurfaceTrain=SurfaceTrain[p]
    krTrain=krTrain[p]
    NN=build_model(best_params_csv.iloc[0].values.tolist())
    NN.summary()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2000,restore_best_weights=True)# Set early stopping, if necessary
    callbacks_list = [es,new_lr]
    history=NN.fit(SurfaceTrain,np.array(krTrain),epochs=2500,validation_split=0.1,callbacks=callbacks_list)
    losshistory.append(history.history['loss'])
    vallosshistory.append(history.history['val_loss'])
    predictions.append(NN.predict(SurfacePool))
    NN.save('./Models/model'+str(i))

In [ ]:
best_params_csv

In [ ]:
N_samples=np.shape(predictions)[1]
predictions=np.array(predictions)
Uncertainty=[np.std(predictions[:,j]) for j in range(N_samples)]# Model uncertainty is the standard deviation of the predictions
Prediction=[np.mean(predictions[:,j]) for j in range(N_samples)]# Model prediction is the averaged prediction

In [ ]:
predict_pool=[]
## An instance of model prediction, note that the current testing datasets are the training datasets
for i in range(Iteration_num):
    Model=load_model("./Models/model"+str(i))
    predict_pool.append(Model.predict(SurfacePool))
predict_pool=np.array(predict_train)

predict_uncertainty=[]
prediction=[]
for i in range(len(SurfacePool)):
    predict_uncertainty.append(np.std(predict_pool[:,i]))
    prediction.append(np.mean(predict_train[:,i]))

In [ ]:
data_input={'PoolID':SurfacePoolID.astype(int),'Prediction':Prediction,'Uncertainty':Uncertainty,'Prediction_Distribution':[predictions[:,i] for i in range(len(SurfacePoolID))]}
SurfacePoolData=pd.DataFrame(data=data_input)

Check uncertainties of roughenss predictions

In [ ]:
SurfacePoolData=SurfacePoolData.set_index('PoolID')

In [ ]:
# Nominate next interested samples following AL framwork
AL_next_ID=list(SurfacePoolData.sort_values("Uncertainty",ascending=False).head(20).index)
AL_next_ID

In [ ]:
Uncertainty_copy=Uncertainty.copy()

In [ ]:
Uncertainty_copy.sort(reverse=True)
plt.plot(Uncertainty_copy,label='Prediction variance')
#plt.plot(Uncertainty[0:20],label='LC surfaces')
plt.legend()
plt.ylabel('Prediction variance')
plt.xlabel('Surface,sorted by uncertainty')